# Settings

## Constants

In [1]:
import os

PATH_DATA = './Dataset'
sensor_data = ['UserInfo.csv', 'Service.csv', 'ContextualFactor.csv', 'Interruptibility.csv']

# Utility Functions

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score


# Load the Dataset into Dataframe

In [3]:
dataframes = {
    filename: pd.read_csv(os.path.join(PATH_DATA, filename)).reset_index(drop=True)
    for filename in sensor_data
}
dfService = dataframes['Service.csv']
dfContextualFactor = dataframes['ContextualFactor.csv']
dfUserInfo = dataframes['UserInfo.csv']
dfInterruptibility = dataframes['Interruptibility.csv']

# Preprocessing

In [4]:
# Select specific columns from dfContextualFactor, dfService, and dfUserInfo DataFrames
# Combine relevant columns to create a unified dataset
dfContextualFactor_selected_columns=dfContextualFactor[['uid','sid','activity1','activity2','activity3','userRoom','userPosition',]]
dfService_selected_columns=dfService[['weekOfExperiment','dayOfWeek','startTime', 'activityInquiry','availabilityInquiry','speechShadowing_1','speechShadowing_2','speechShadowing_3','speechShadowing_4','speechShadowing_5','continue-to-nextInquiry_1','continue-to-nextInquiry_2','continue-to-nextInquiry_3','continue-to-nextInquiry_4','endTime','endType']]
dfInterruptibility_selected_columns=dfInterruptibility[['SHORT_INTERACTION_availability', 'LONG_INTERACTION_availability']]

## Create binary columns for interaction types
## SHORT_INTERACTION: True if availabilityInquiry is not NaN
# dfCombinedAll['SHORT_INTERACTION_availability'] = dfCombinedAll['availabilityInquiry'].notna()
## LONG_INTERACTION: True if continue-to-nextInquiry_1 is not NaN
# dfCombinedAll['LONG_INTERACTION_availability'] = dfCombinedAll['continue-to-nextInquiry_1'].notna()

# ! For those who want to redefine LONG_INTERACTION with thresholds longer than 3 minutes,
# you can use the following columns:
# 5 minutes  => use continue-to-nextInquiry_2
# 7 minutes  => use continue-to-nextInquiry_3
# 9 minutes  => use continue-to-nextInquiry_4


dfCombinedAll=pd.concat([dfContextualFactor_selected_columns, dfService_selected_columns,dfInterruptibility_selected_columns], axis=1)

In [5]:
# Concatenate and merge the all columns from dfUserInfo (dfUserInfo.csv)
dfUserInfo_selected_columns = dfUserInfo[['uid', 'settingType', 'speakerRoom', 'speakerPosition']]
dfCombinedAll = pd.merge(dfCombinedAll, dfUserInfo_selected_columns, on='uid', how='left')

## Position Processing

In [6]:
# Define function to calculate proximity between user and speaker
def calculate_proximity(row):
    # Return 0 if user and speaker are in different rooms
    if row['userRoom'] != row['speakerRoom']:
        return 0
    # If in the same room
    elif row['userPosition'] == row['speakerPosition']:
        return 2  # Same position
    else:
        return 1  # Different positions (including missing position)

# Apply proximity calculation to create a new 'proximity' column
dfCombinedAll['proximity'] = dfCombinedAll.apply(calculate_proximity, axis=1)

print(dfCombinedAll[['userRoom', 'userPosition', 'speakerRoom', 'speakerPosition', 'proximity']].head(10))


      userRoom userPosition speakerRoom speakerPosition  proximity
0     Bed Room          Bed    Bed Room            Desk          1
1    Rest Room          NaN    Bed Room            Desk          0
2  Living Room          NaN    Bed Room            Desk          0
3  Living Room          NaN    Bed Room            Desk          0
4     Bed Room          Bed    Bed Room            Desk          1
5     Bed Room          Bed    Bed Room            Desk          1
6     Bed Room          Bed    Bed Room            Desk          1
7     Bed Room         Desk    Bed Room            Desk          2
8     Bed Room          Bed    Bed Room            Desk          1
9     Bed Room          Bed    Bed Room            Desk          1


## Activity and Time Processing

In [7]:
# Process activity columns for one-hot encoding
activity_cols = ['activity1', 'activity2', 'activity3']
df_activity = dfContextualFactor[activity_cols].copy()

# Get unique activities across all activity columns, excluding NaN
all_unique_activities = pd.unique(df_activity.values.ravel())
all_unique_activities = [x for x in all_unique_activities if pd.notna(x)]

# Create a DataFrame for one-hot encoding of activities
dfActivity_one_hot_encoding = pd.DataFrame(0, index=df_activity.index, columns=['act_' + str(val) for val in all_unique_activities])

# Perform one-hot encoding for each activity column
for col in activity_cols:
    for val in all_unique_activities:
        dfActivity_one_hot_encoding['act_' + str(val)] |= (df_activity[col] == val).astype(int)

# Concatenate one-hot encoded activity columns to dfCombinedAll
dfActivity_one_hot_encoding
dfCombinedAll = pd.concat([dfCombinedAll, dfActivity_one_hot_encoding], axis=1)

In [8]:
# Convert startTime to datetime and extract total minutes since midnight
dfCombinedAll['startTime'] = pd.to_datetime(dfCombinedAll['startTime'], format='%H:%M:%S', errors='coerce')
dfCombinedAll['minute'] = dfCombinedAll['startTime'].dt.hour * 60 + dfCombinedAll['startTime'].dt.minute

# Map days of the week to numerical values (MON=0, TUE=1, ..., SUN=6)
day_map = {'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3, 'FRI': 4, 'SAT': 5, 'SUN': 6}
dfCombinedAll['dayOfWeek'] = dfCombinedAll['dayOfWeek'].map(day_map)

# Bin minutes into 30-minute intervals for temporal analysis
dfCombinedAll['minute_bin'] = (dfCombinedAll['minute'] // 30).astype(int)

In [9]:
dfCombinedAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2901 entries, 0 to 2900
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   uid                                       2901 non-null   int64         
 1   sid                                       2901 non-null   int64         
 2   activity1                                 2901 non-null   object        
 3   activity2                                 112 non-null    object        
 4   activity3                                 3 non-null      object        
 5   userRoom                                  2901 non-null   object        
 6   userPosition                              2414 non-null   object        
 7   weekOfExperiment                          2901 non-null   int64         
 8   dayOfWeek                                 2901 non-null   int64         
 9   startTime                     

# Feature Extraction

In [10]:
# Select features for response prediction
dfFeatresForResponse = dfCombinedAll[['uid',
    'act_Taking a Nap / Sleeping','act_Hygiene','act_Eating','act_Using Media','act_Social Interaction',
    'act_Returning from Outside / Other Rooms','act_Studying / Working','act_Others','act_House Chores',
    'act_Self Caring','act_Visiting Outside / Other Rooms','act_Resting',
    'settingType','userRoom','userPosition','speakerRoom','speakerPosition',
    'minute_bin','dayOfWeek','SHORT_INTERACTION_availability','LONG_INTERACTION_availability']].copy()

# Encode categorical columns using LabelEncoder
categorical_columns = ['settingType', 'userRoom', 'userPosition', 'speakerRoom', 'speakerPosition', 'minute_bin']
label_encoders = defaultdict(LabelEncoder)

# Apply label encoding to each categorical column
for col in categorical_columns:
    dfFeatresForResponse[col] = label_encoders[col].fit_transform(dfFeatresForResponse[col])

# Create a copy of the encoded data for further processing
encoded_data = dfFeatresForResponse.copy()


encoded_data.info()
encoded_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2901 entries, 0 to 2900
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype
---  ------                                    --------------  -----
 0   uid                                       2901 non-null   int64
 1   act_Taking a Nap / Sleeping               2901 non-null   int64
 2   act_Hygiene                               2901 non-null   int64
 3   act_Eating                                2901 non-null   int64
 4   act_Using Media                           2901 non-null   int64
 5   act_Social Interaction                    2901 non-null   int64
 6   act_Returning from Outside / Other Rooms  2901 non-null   int64
 7   act_Studying / Working                    2901 non-null   int64
 8   act_Others                                2901 non-null   int64
 9   act_House Chores                          2901 non-null   int64
 10  act_Self Caring                           2901 non-null   in

,uid,act_Taking a Nap / Sleeping,act_Hygiene,act_Eating,act_Using Media,act_Social Interaction,act_Returning from Outside / Other Rooms,act_Studying / Working,act_Others,act_House Chores,...,act_Resting,settingType,userRoom,userPosition,speakerRoom,speakerPosition,minute_bin,dayOfWeek,SHORT_INTERACTION_availability,LONG_INTERACTION_availability
0,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,12,0,True,False
1,1,0,1,0,0,0,0,0,0,0,...,0,0,7,3,0,1,15,0,False,False
2,1,0,0,1,0,0,0,0,0,0,...,0,0,5,3,0,1,12,1,False,False
3,1,0,0,1,0,0,0,0,0,0,...,0,0,5,3,0,1,13,1,False,False
4,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,18,1,True,False


## Label: SHORT_INTERACTION

In [11]:
# Label distribution before balancing
print(encoded_data['SHORT_INTERACTION_availability'].value_counts())

SHORT_INTERACTION_availability
True     2158
False     743
Name: count, dtype: int64


### Model Building and LOSO CV

In [12]:
# Prepare features (X) and target variables (y) for SHORT_INTERACTION
X = encoded_data.drop(columns=['SHORT_INTERACTION_availability', 'LONG_INTERACTION_availability', 'uid'], axis=1) # Drop target and unrelated columns
y = encoded_data['SHORT_INTERACTION_availability'] # Target variable
groups = encoded_data['uid'] # Group by user ID for Leave-One-Group-Out CV

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models1 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results1_logo = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)
    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1_logo[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2104, number of negative: 2104
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 4208, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2088, number of negative: 2088
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 4176, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is d

...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6539924	total: 1.05ms	remaining: 104ms
1:	learn: 0.6247162	total: 1.89ms	remaining: 92.5ms
2:	learn: 0.5950981	total: 2.63ms	remaining: 85.1ms
3:	learn: 0.5743052	total: 3.51ms	remaining: 84.2ms
4:	learn: 0.5560487	total: 4.38ms	remaining: 83.3ms
5:	learn: 0.5401743	total: 5.13ms	remaining: 80.4ms
6:	learn: 0.5206125	total: 6.01ms	remaining: 79.9ms
7:	learn: 0.5042045	total: 6.86ms	remaining: 78.9ms
8:	learn: 0.4910786	total: 7.66ms	remaining: 77.5ms
9:	learn: 0.4796726	total: 8.49ms	remaining: 76.4ms
10:	learn: 0.4683951	total: 9.41ms	remaining: 76.1ms
11:	learn: 0.4590041	total: 10.7ms	remaining: 78.1ms
12:	learn: 0.4510206	total: 11.8ms	remaining: 79ms
13:	learn: 0.4427894	total: 12.7ms	remaining: 78ms
14:	learn: 0.4381560	total: 13.7ms	remaining: 77.4ms
15:	learn: 0.4324047	total: 14.5ms	remaining: 76.1ms
16:	learn: 0.4268654	total: 15.3ms	remaining: 74.9ms
17:	learn: 0.4219865	total: 16.2ms	remaining: 73.9ms
18:	learn: 0.4181830	total: 17.1ms	remaining: 72.9ms
19:	lear

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6564588	total: 966us	remaining: 95.7ms
1:	learn: 0.6235117	total: 1.84ms	remaining: 90.3ms
2:	learn: 0.5928634	total: 2.58ms	remaining: 83.3ms
3:	learn: 0.5715604	total: 3.43ms	remaining: 82.3ms
4:	learn: 0.5515084	total: 4.23ms	remaining: 80.4ms
5:	learn: 0.5328525	total: 5.23ms	remaining: 81.9ms
6:	learn: 0.5138486	total: 6.35ms	remaining: 84.3ms
7:	learn: 0.4988853	total: 7.22ms	remaining: 83ms
8:	learn: 0.4844758	total: 8ms	remaining: 80.9ms
9:	learn: 0.4771154	total: 8.55ms	remaining: 76.9ms
10:	learn: 0.4690129	total: 9.34ms	remaining: 75.6ms
11:	learn: 0.4598755	total: 10.2ms	remaining: 75.1ms
12:	learn: 0.4507046	total: 11.1ms	remaining: 74.5ms
13:	learn: 0.4430011	total: 12ms	remaining: 73.6ms
14:	learn: 0.4366333	total: 12.9ms	remaining: 72.9ms
15:	learn: 0.4309415	total: 13.7ms	remaining: 72.2ms
16:	learn: 0.4244507	total: 14.6ms	remaining: 71.1ms
17:	learn: 0.4196403	total: 15.4ms	remaining: 70.3ms
18:	learn: 0.4150978	total: 16.2ms	remaining: 69.2ms
19:	learn: 

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\3270779774.py:40: FutureWarning: Series.ravel is d

### K-Fold CV

In [13]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results1_kfold = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1_kfold[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1716, number of negative: 1716
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000251 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6529720	total: 797us	remaining: 78.9ms
1:	learn: 0.6170222	total: 1.74ms	remaining: 85.1ms
2:	learn: 0.5874868	total: 2.5ms	remaining: 80.8ms
3:	learn: 0.5622815	total: 3.26ms	remaining: 78.1ms
4:	learn: 0.5436234	total: 3.99ms	remaining: 75.7ms
5:	learn: 0.5282429	total: 4.75ms	remaining: 74.3ms
6:	learn: 0.5077982	total: 5.49ms	remaining: 72.9ms
7:	learn: 0.4919153	total: 6.23ms	remaining: 71.7ms
8:	learn: 0.4823170	total: 7.03ms	remaining: 71ms
9:	learn: 0.4678202	total: 7.81ms	remaining: 70.3ms
10:	learn: 0.4587949	total: 8.57ms	remaining: 69.4ms
11:	learn: 0.4504857	total: 9.31ms	remaining: 68.3ms
12:	learn: 0.4399476	total: 10.1ms	remaining: 67.6ms
13:	learn: 0.4298470	total: 10.9ms	remaining: 66.7ms
14:	learn: 0.4216719	total: 11.6ms	remaining: 65.7ms
15:	learn: 0.4143371	total: 12.3ms	remaining: 64.8ms
16:	learn: 0.4103875	total: 13.1ms	remaining: 64ms
17:	learn: 0.4067765	total: 13.9ms	remaining: 63.1ms
18:	learn: 0.4027866	total: 14.6ms	remaining: 62.4ms
19:	learn

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


50:	learn: 0.3556333	total: 39.6ms	remaining: 38.1ms
51:	learn: 0.3550356	total: 40.4ms	remaining: 37.3ms
52:	learn: 0.3545033	total: 41.2ms	remaining: 36.6ms
53:	learn: 0.3533526	total: 42.1ms	remaining: 35.9ms
54:	learn: 0.3521534	total: 42.9ms	remaining: 35.1ms
55:	learn: 0.3513438	total: 43.7ms	remaining: 34.3ms
56:	learn: 0.3509386	total: 44.5ms	remaining: 33.6ms
57:	learn: 0.3500363	total: 45.4ms	remaining: 32.8ms
58:	learn: 0.3491584	total: 46.1ms	remaining: 32.1ms
59:	learn: 0.3483795	total: 47ms	remaining: 31.4ms
60:	learn: 0.3477386	total: 47.9ms	remaining: 30.6ms
61:	learn: 0.3465586	total: 49.1ms	remaining: 30.1ms
62:	learn: 0.3461872	total: 50.1ms	remaining: 29.4ms
63:	learn: 0.3452851	total: 51.2ms	remaining: 28.8ms
64:	learn: 0.3444363	total: 52.2ms	remaining: 28.1ms
65:	learn: 0.3431280	total: 53ms	remaining: 27.3ms
66:	learn: 0.3424332	total: 53.7ms	remaining: 26.5ms
67:	learn: 0.3413738	total: 54.5ms	remaining: 25.7ms
68:	learn: 0.3406611	total: 55.3ms	remaining: 24.8

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\2766920084.py:23: FutureWarning: Series.ravel is d

## Label: LONG_INTERACTION

In [14]:
# Label distribution before balancing
print(encoded_data['LONG_INTERACTION_availability'].value_counts())

LONG_INTERACTION_availability
False    1468
True     1433
Name: count, dtype: int64


### Model building and LOSO CV

In [15]:
# Prepare features (X) and target variable (y) for predicting LONG_INTERACTION
X = encoded_data.drop(columns=['LONG_INTERACTION_availability', 'SHORT_INTERACTION_availability','uid'], axis=1)
y = encoded_data['LONG_INTERACTION_availability']
groups = encoded_data['uid']

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models2 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results2_logo = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2_logo[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1425, number of negative: 1425
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 2850, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1365, number of negative: 1365
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000248 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 92
[LightGBM] [Info] Number of data points in the train set: 2730, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


73:	learn: 0.5030455	total: 55.1ms	remaining: 19.3ms
74:	learn: 0.5026702	total: 56ms	remaining: 18.7ms
75:	learn: 0.5022147	total: 57ms	remaining: 18ms
76:	learn: 0.5017477	total: 57.7ms	remaining: 17.2ms
77:	learn: 0.5014188	total: 58.5ms	remaining: 16.5ms
78:	learn: 0.5012624	total: 59.2ms	remaining: 15.7ms
79:	learn: 0.5006921	total: 59.9ms	remaining: 15ms
80:	learn: 0.5003697	total: 60.6ms	remaining: 14.2ms
81:	learn: 0.5000732	total: 61.3ms	remaining: 13.5ms
82:	learn: 0.4993752	total: 62.1ms	remaining: 12.7ms
83:	learn: 0.4991525	total: 63ms	remaining: 12ms
84:	learn: 0.4987317	total: 63.9ms	remaining: 11.3ms
85:	learn: 0.4983181	total: 64.7ms	remaining: 10.5ms
86:	learn: 0.4977586	total: 65.6ms	remaining: 9.81ms
87:	learn: 0.4974926	total: 66.3ms	remaining: 9.05ms
88:	learn: 0.4972103	total: 67.1ms	remaining: 8.29ms
89:	learn: 0.4967334	total: 67.8ms	remaining: 7.53ms
90:	learn: 0.4959122	total: 68.5ms	remaining: 6.77ms
91:	learn: 0.4951555	total: 69.2ms	remaining: 6.02ms
92:	l

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6754471	total: 1.01ms	remaining: 99.8ms
1:	learn: 0.6566711	total: 1.79ms	remaining: 87.8ms
2:	learn: 0.6439721	total: 2.65ms	remaining: 85.6ms
3:	learn: 0.6308258	total: 3.51ms	remaining: 84.2ms
4:	learn: 0.6211945	total: 4.24ms	remaining: 80.5ms
5:	learn: 0.6113630	total: 5.2ms	remaining: 81.5ms
6:	learn: 0.6042297	total: 6.06ms	remaining: 80.5ms
7:	learn: 0.5964287	total: 6.99ms	remaining: 80.3ms
8:	learn: 0.5908933	total: 7.91ms	remaining: 80ms
9:	learn: 0.5838353	total: 8.77ms	remaining: 79ms
10:	learn: 0.5783850	total: 9.78ms	remaining: 79.1ms
11:	learn: 0.5740401	total: 10.7ms	remaining: 78.4ms
12:	learn: 0.5704543	total: 11.5ms	remaining: 77.2ms
13:	learn: 0.5676013	total: 12.3ms	remaining: 75.4ms
14:	learn: 0.5644495	total: 13ms	remaining: 73.6ms
15:	learn: 0.5612579	total: 13.7ms	remaining: 72ms
16:	learn: 0.5581067	total: 14.5ms	remaining: 70.6ms
17:	learn: 0.5552219	total: 15.2ms	remaining: 69ms
18:	learn: 0.5533608	total: 15.9ms	remaining: 67.8ms
19:	learn: 0.5

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\582202991.py:40: FutureWarning: Series.ravel is depre

### K-Fold CV

In [16]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results2_kfold = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2_kfold[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1188, number of negative: 1188
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000184 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 2376, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1201, number of negative: 1201
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000172 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 2402, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6709850	total: 800us	remaining: 79.2ms
1:	learn: 0.6530890	total: 1.57ms	remaining: 76.8ms
2:	learn: 0.6385725	total: 2.25ms	remaining: 72.9ms
3:	learn: 0.6239373	total: 2.96ms	remaining: 71ms
4:	learn: 0.6103134	total: 3.62ms	remaining: 68.8ms
5:	learn: 0.6008130	total: 4.24ms	remaining: 66.4ms
6:	learn: 0.5925854	total: 4.86ms	remaining: 64.5ms
7:	learn: 0.5861364	total: 5.48ms	remaining: 63.1ms
8:	learn: 0.5789559	total: 6.2ms	remaining: 62.6ms
9:	learn: 0.5715430	total: 6.9ms	remaining: 62.1ms
10:	learn: 0.5669046	total: 7.51ms	remaining: 60.7ms
11:	learn: 0.5635532	total: 8.18ms	remaining: 60ms
12:	learn: 0.5596333	total: 8.8ms	remaining: 58.9ms
13:	learn: 0.5562383	total: 9.45ms	remaining: 58.1ms
14:	learn: 0.5515176	total: 10.1ms	remaining: 57.3ms
15:	learn: 0.5482678	total: 10.8ms	remaining: 56.6ms
16:	learn: 0.5446591	total: 11.4ms	remaining: 55.8ms
17:	learn: 0.5421223	total: 12ms	remaining: 54.8ms
18:	learn: 0.5396655	total: 12.7ms	remaining: 54.2ms
19:	learn: 0.

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6715660	total: 841us	remaining: 83.3ms
1:	learn: 0.6555662	total: 1.61ms	remaining: 79ms
2:	learn: 0.6381368	total: 2.31ms	remaining: 74.6ms
3:	learn: 0.6248515	total: 2.93ms	remaining: 70.3ms
4:	learn: 0.6128915	total: 3.56ms	remaining: 67.6ms
5:	learn: 0.6055659	total: 4.24ms	remaining: 66.4ms
6:	learn: 0.5984308	total: 4.94ms	remaining: 65.6ms
7:	learn: 0.5916192	total: 5.64ms	remaining: 64.9ms
8:	learn: 0.5851955	total: 6.29ms	remaining: 63.6ms
9:	learn: 0.5796052	total: 6.94ms	remaining: 62.5ms
10:	learn: 0.5756915	total: 7.56ms	remaining: 61.2ms
11:	learn: 0.5702822	total: 8.21ms	remaining: 60.2ms
12:	learn: 0.5645748	total: 8.84ms	remaining: 59.2ms
13:	learn: 0.5608572	total: 9.51ms	remaining: 58.4ms
14:	learn: 0.5560588	total: 10.2ms	remaining: 57.6ms
15:	learn: 0.5532133	total: 10.9ms	remaining: 57.1ms
16:	learn: 0.5492534	total: 11.6ms	remaining: 56.4ms
17:	learn: 0.5464919	total: 12.2ms	remaining: 55.7ms
18:	learn: 0.5429196	total: 12.9ms	remaining: 55.2ms
19:	le

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23592\1454448026.py:23: FutureWarning: Series.ravel is d

In [ ]:
from tabulate import tabulate

# Models list
models = [
    'Random Forest', 'Gradient Boosting', 'XGBoost',
    'LightGBM', 'CatBoost', 'SVM', 'Dummy'
]

# Prepare table data
table_data = []
for model in models:
    row = [model]
    
    # Shell 1: SHORT_INTERACTION, Leave-One-Group-Out
    row.append(results1_logo.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results1_logo.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    # Shell 3: LONG_INTERACTION, Leave-One-Group-Out
    row.append(results2_logo.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results2_logo.get(model, {}).get('Average F1-Score (macro)', 0.0))

    # Shell 2: SHORT_INTERACTION, 5-fold
    row.append(results1_kfold.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results1_kfold.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    # Shell 4: LONG_INTERACTION, 5-fold
    row.append(results2_kfold.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results2_kfold.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    table_data.append(row)

# Define headers
headers = [
    'Model',
    'LOSO CV\nShort\nInteraction\nAccuracy', 'LOSO CV\nShort\nInteraction\nF1',
    'LOSO CV\nLong\nInteraction\nAccuracy', 'LOSO CV\nLong\nInteraction\nF1',
    '5-fold CV\nShort\nInteraction\nAccuracy', '5-fold CV\nShort\nInteraction\nF1',
    '5-fold CV\nLong\nInteraction\nAccuracy', '5-fold CV\nLong\nInteraction\nF1'
]

# Print the table
print("Machine learning model performance")
print(tabulate(table_data, headers=headers, tablefmt='fancy_grid', floatfmt='.4f', numalign="decimal"))


Machine learning model performance
╒═══════════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ Model             │       LOSO CV │       LOSO CV │       LOSO CV │       LOSO CV │     5-fold CV │     5-fold CV │     5-fold CV │     5-fold CV │
│                   │         Short │         Short │          Long │          Long │         Short │         Short │          Long │          Long │
│                   │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │
│                   │      Accuracy │            F1 │      Accuracy │            F1 │      Accuracy │            F1 │      Accuracy │            F1 │
╞═══════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╡
│ Random Forest     │         0.834 │         0.738 │         0.6